In [1]:
%%HTML
<style>
.container { width:100% }
</style>

# ETL

## Import der benötigten Bibliotheken

In [2]:
import nltk                            # bilden der Bigrams
import operator                        # sortieren des Dictionarys
import re                              # Regular Expressions, Entfernen von Zeichen
import pandas    as pd                 # .txt konvertieren, .csv einlesen und abspeichern
from pymongo     import MongoClient    # Verbindung und Interaktion mit mongoDB
from string      import ascii_letters  # Ascii-Buchstaben, die erhalten bleiben
from pathlib     import Path           # Prüfen, ob Datei vorhanden ist
from nltk.corpus import stopwords      # Füllworte


## Laden der Sätze und falls notwendig Konvertierung der .txt-Datenquellen in .csv-Dateien

In [3]:
if Path('sentences_de.csv').is_file():
    sentences_csv = pd.read_csv ('sentences_de.csv', names=['id', 'sentence'], header = 0)
elif Path('deu_news_2015_3M-sentences.txt').is_file():
    sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', names=['id', 'sentence'], header = 0, delimiter='\t')
    sentences_csv.to_csv ('sentences_de.csv', index=None)
else:
    print("Die benötigten Daten existieren nicht. Bitte lade sie von folgender Seite runter: https://www.kaggle.com/rtatman/3-million-german-sentences")

## Laden der Stopwords

In [4]:
nltk.download('stopwords')
stop = set(stopwords.words('german'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Luisa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Entfernen der Stopwords und allen Zeichen, die keine Ascii-Buchstaben sind

In [5]:
def remove_stopwords_and_nonascii(sentences):
    queue = []
    for sentence in sentences:
        just_ascii = re.sub(r"[^{}]".format(ascii_letters + 'äöüÄÖÜß'), ' ', sentence) 
        result = ' '.join([i for i in just_ascii.lower().split() if i not in stop and i != ' '])
        if result != ' ': queue.append(result)
    return queue

In [6]:
nostop = remove_stopwords_and_nonascii(sentences_csv['sentence'])
nostop = pd.DataFrame(nostop, columns=['sentence'])
nostop.to_csv('sentences_preprocessed.csv', index=False)

## Verbindung mit mongoDB

In [7]:
connection_string      = "mongodb://localhost:27017"
db                     = MongoClient(connection_string)
sentences              = db.dw.sentences
sentences_preprocessed = db.dw.sentences_preprocessed
bigrams                = db.dw.bigrams

## Sentences und Stopwords der Database in mongoDB hinzufügen

`save_to_mongo()` speichert Listen von Dictionarys oder DataFrames in einer Collection. Dafür kann zunächst die Collection gelöscht werden (`drop_collection = True`). Sonst werden die Daten zusätzlich in die Collection eingefügt.

In [8]:
def save_to_mongo(collection, data, drop_collection):
    if drop_collection:
        collection.drop()
    if isinstance(data, pd.DataFrame): data = data.to_dict('records')
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [9]:
save_to_mongo(sentences, sentences_csv, True)

2646182 rows are saved to "sentences" collection successfully!


In [10]:
save_to_mongo(sentences_preprocessed, nostop, True)

2646182 rows are saved to "sentences_preprocessed" collection successfully!


# Analyse

## Extraktion der Daten
`get_column_of_collection()` liest alle Datensätze aus einer Spalte einer Collection und gibt eine Liste mit diesen Datensätzen zurück.

In [11]:
def get_column_of_collection(collection, column):
    documents = []
    cursor = collection.find({})
    for document in cursor:
        documents.append(document[column])
    return documents

In [12]:
extracted_documents = get_column_of_collection(sentences_preprocessed, 'sentence')

## Finden der häufisten Bigrams (= 2-Wort-Kombinationen)

`count_bigrams()` nimmt eine Liste von Sätzen, bildet alle Bigrams, und speichert diese mit ihrer Häufigkeit in einem Dictionary.

In [13]:
def count_bigrams(data):
    bigram_dict = dict()
    for i in range(len(data)):
        text = data[i]
        bigrams = list(nltk.bigrams(text.split()))
        for bigram in bigrams:
            if bigram in bigram_dict:
                bigram_dict[bigram] += 1
            else:
                bigram_dict[bigram] = 1
    return bigram_dict

`get_k_most_frequent()` nimmt ein zuvor mit `count_bigrams()`erstelltes Dictionary, sortiert es nach den Haüfigkeiten, und gibt die k Elemente mit der größten Häufigkeit zurück.

In [14]:
def get_k_most_frequent(dictionary, k):
    sorted_dictionary =  dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))
    result = sorted_dictionary.items()
    return list(result)[:k]

`pretty_print_most_frequent()` nimmt die von `get_k_most_frequent()` ausgegebenen Datensätze und gibt sie schön formatiert aus.

In [15]:
def pretty_print_most_frequent(data):
    for i in range(len(data)):
        words, frequency = data[i]
        print('Platz ', i+1, ': Die Wortkombination "', ' '.join(words), \
              '" kommt in den Daten ', frequency, ' mal vor.', sep='')

`bigrams_to_dict()` bereitet die Bigrams so vor, dass sie in einer Collection gespeichert werden können.

In [16]:
def bigrams_to_dict(bigrams):
    result = []
    for item in bigrams:
        dictionary = {'bigram': item, 'frequency': bigrams[item]}
        result.append(dictionary)
    return result

`get_k_most_frequent_bigrams()` ruft die zuvor implementierten Funktionen auf. Zunächst werden die Bigrams gebildet und gezählt. Diese werden dann formatiert und in der Datenbank gespeichert. Anschließend werden die k häufigsten Bigrams schön ausgedruckt und schließlich auch zurückgegeben.

In [17]:
def get_k_most_frequent_bigrams(data, k, collection_to_save):
    bigram_dict   = count_bigrams(data)
    saveable_dict = bigrams_to_dict(bigram_dict)
    save_to_mongo(collection_to_save, saveable_dict, True)
    most_frequent = get_k_most_frequent(bigram_dict, k)
    pretty_print_most_frequent(most_frequent)
    return most_frequent

In [18]:
%%time
most_frequent = get_k_most_frequent_bigrams(extracted_documents, 10, bigrams)

13480027 rows are saved to "bigrams" collection successfully!
Platz 1: Die Wortkombination "e mail" kommt in den Daten 15108 mal vor.
Platz 2: Die Wortkombination "millionen euro" kommt in den Daten 11995 mal vor.
Platz 3: Die Wortkombination "milliarden euro" kommt in den Daten 8993 mal vor.
Platz 4: Die Wortkombination "antwort schreiben" kommt in den Daten 7553 mal vor.
Platz 5: Die Wortkombination "seit jahren" kommt in den Daten 6488 mal vor.
Platz 6: Die Wortkombination "new york" kommt in den Daten 5108 mal vor.
Platz 7: Die Wortkombination "us dollar" kommt in den Daten 4957 mal vor.
Platz 8: Die Wortkombination "mail adresse" kommt in den Daten 4883 mal vor.
Platz 9: Die Wortkombination "orf at" kommt in den Daten 4797 mal vor.
Platz 10: Die Wortkombination "vergangenen jahr" kommt in den Daten 4776 mal vor.
Wall time: 57min 41s


# ToDos
- Einlesen von csv ohne Konvertierung ✓
- Einfügen von sentences_preprocessed in DB ohne Umweg über DataFrame und .csv ✓
- Spaltenname ohne Umbenennung ✓
- Daten für Analyse aus DB ziehen ✓
- Das Ergebnis (komplettes Bigram-Dict?) in der DB speichern ✓
- Weitere Analysen?
- Satzzeichen bei Stopwörtern hizufügen ✓